In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from datetime import datetime
import wandb
from qrytool import load_data_into_dataframe, insert_dataframe_into_table
from geopy.distance import geodesic
import pandas as pd
import concurrent.futures
import os
import ray
from tqdm.notebook import tqdm
from geotool import replace_address_sido


def calculate_min_distance(row, df_subway):
    lat1, lon1 = row['위도'], row['경도']
    min_distance = float('inf')
    nearest_station = None
    row_region = row['지역'].split()[0]

    # Filter df_subway by matching the first word of '지역' column
    filtered_df_subway = df_subway[df_subway['지역'].apply(lambda x: x.split()[0]) == row_region]

    if not filtered_df_subway.empty:
        for _, subway_row in filtered_df_subway.iterrows():
            lat2, lon2 = subway_row['lati'], subway_row['longi']
            distance = geodesic((lat1, lon1), (lat2, lon2)).meters

            if distance < min_distance:
                min_distance = distance
                nearest_station = subway_row
    else:
        # If no matching rows are found
        return {
            'index': row['index'],
            '최단지하철역': '없음',
            '역과거리': 10000000,
            '역사명': '없음',
            '근접노선수': 0,
            '역lati': 51.5074,
            '역longi': 0.1278,
            '도보거리': False
        }

    return {
        'index': row['index'],
        '최단지하철역': nearest_station['역사명'],
        '역과거리': min_distance,
        '역사명': nearest_station['역사명'],
        '근접노선수': nearest_station['근접노선수'],
        '역lati': nearest_station['lati'],
        '역longi': nearest_station['longi'],
        '도보거리': min_distance <= 500
    }


def prepare_subway_data(df_subway):
    def get_region(address):
        region = replace_address_sido(address)
        return ' '.join(region.split()[:2])

    df_subway['지역'] = df_subway.apply(
        lambda row: get_region('대구시 ' + row['역사도로명주소']) if row['운영기관명'] == '대구교통공사' else get_region(row['역사도로명주소']),
        axis=1
    )
    return df_subway


def process_region_naver_cortar_addr(dongne):
    words = dongne.split()
    if len(words) > 0 and words[0].endswith('시'):
        return ' '.join([words[0][:-1]] + words[1:2])
    else:
        region = replace_address_sido(dongne)
        return ' '.join(region.split()[:2])

In [ ]:


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_colwidth", None)

qry = """
SELECT
    complex_name 아파트명,
    cortar_address 동네,
    real_estate_type_name 건물종류,
    latitude 위도,
    longitude 경도,
    CAST(REPLACE(pyeong_content, '공급 ', '') AS DECIMAL) AS 면적,
    floor 층,
    formatted_trade_year_month 거래년월일,
    total_household_count 가구수,
    total_building_count 동수,
    use_approve_ymd 사용승인년월일,
    deal_price 거래가
FROM train_base_data;
"""

df = load_data_into_dataframe(qry)

df['거래년월일'] = pd.to_datetime(df['거래년월일'], errors='coerce')
df['사용승인년월일'] = pd.to_datetime(df['사용승인년월일'], errors='coerce')

# Process '동네' column
df['지역'] = df['동네'].apply(process_region_naver_cortar_addr)


# 오늘 날짜를 기준으로 건물의 나이 계산
today = datetime.now()
df['나이'] = (today - df['사용승인년월일']).dt.days / 365.25  # 일수를 년도로 변환

# 인덱스 추가 (결과 병합을 위해)
df.reset_index(inplace=True)
df = df[~df['지역'].str.contains('제주')]

# 지하철 거리 연관성 학습 데이터 생성


기존 학습된 데이터가 있을 경우 다시 계산하지 않고 계산하여 저장한 파일을 로드하여 진행하면 됨.


In [ ]:
# 순차적으로 처리 실행
batch_size = 300
num_cores = 23
# 결과 저장할 CSV 파일 초기화
output_file = 'output.csv'


if not ray.is_initialized():
    ray.init(num_cpus=23)


@ray.remote
def process_batch(batch, df_subway):
    results = []
    for _, row in batch.iterrows():
        result = calculate_min_distance(row, df_subway)
        results.append(result)
    return results


# 데이터 로드
file_path = "./전국_도시철도역사정보_좌표포함.xlsx"
df_subway = pd.read_excel(file_path)
df_subway = prepare_subway_data(df_subway)

# 처리된 인덱스 추적
completed_indices = set()
if os.path.exists(output_file):
    processed_results = pd.read_csv(output_file)
    completed_indices = set(processed_results['index'])
else:
    # 결과 파일에 헤더 작성
    pd.DataFrame(columns=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리']).to_csv(output_file, index=False)

# 처리되지 않은 행들만 선택
remaining_rows = df[~df['index'].isin(completed_indices)]
total_rows = len(remaining_rows)

# Custom bar format for tqdm
bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}] {percentage:3.0f}%'

# Process the data in batches
with tqdm(total=total_rows, desc="Processing rows", bar_format=bar_format) as pbar:
    futures = []
    start_index = 0

    while start_index < total_rows or futures:
        # Submit tasks until the core limit is reached
        while len(futures) < num_cores and start_index < total_rows:
            batch = remaining_rows.iloc[start_index:start_index + batch_size]
            future = process_batch.remote(batch, df_subway)
            futures.append(future)
            start_index += batch_size

        # Wait for the first available future to complete
        if futures:
            ready_futures, remaining_futures = ray.wait(futures, num_returns=1)
            futures = remaining_futures

            for ready_future in ready_futures:
                result = ray.get(ready_future)
                result_df = pd.DataFrame(result)
                result_df.to_csv(output_file, mode='a', header=False, index=False)
                pbar.update(batch_size)

# Shut down Ray
ray.shutdown()

Processing rows:   0%|          | 0/10181332 [00:00<?, ?it/s]   0%

In [ ]:
output_file = 'output.csv'

# 결과 파일에서 최종 DataFrame 로드 및 원래 DataFrame과 병합
# processed_results = pd.read_csv(output_file, header=None, names=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리'])
processed_results = pd.read_csv(output_file, header=None, names=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리'])
df_final = df.merge(processed_results, on='index')

/tmp/ipykernel_3198134/2703775935.py:5: DtypeWarning: Columns (0,2,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_results = pd.read_csv(output_file, header=None, names=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리'])


In [ ]:
df.shape

(10181332, 15)

In [ ]:
df_final.shape

(10115797, 22)

In [ ]:
df.head()

,index,아파트명,동네,건물종류,위도,경도,면적,층,거래년월일,가구수,동수,사용승인년월일,거래가,지역,나이
0,0,꿈의숲코오롱하늘채,서울시 성북구 장위동,아파트,37.619397,127.04663,115.07,30.0,2024-04-29,513,5,2017-10-30,100800.0,서울 성북구,6.699521
1,1,꿈의숲코오롱하늘채,서울시 성북구 장위동,아파트,37.619397,127.04663,115.07,20.0,2024-04-20,513,5,2017-10-30,98000.0,서울 성북구,6.699521
2,2,꿈의숲코오롱하늘채,서울시 성북구 장위동,아파트,37.619397,127.04663,115.07,10.0,2024-02-25,513,5,2017-10-30,95000.0,서울 성북구,6.699521
3,3,꿈의숲코오롱하늘채,서울시 성북구 장위동,아파트,37.619397,127.04663,115.07,4.0,2023-07-10,513,5,2017-10-30,90000.0,서울 성북구,6.699521
4,4,꿈의숲코오롱하늘채,서울시 성북구 장위동,아파트,37.619397,127.04663,115.07,19.0,2023-01-11,513,5,2017-10-30,80000.0,서울 성북구,6.699521


In [ ]:
df_final.head()
df_final[df_final['역과거리'] > 10000].tail()

,index,아파트명,동네,건물종류,위도,경도,면적,층,거래년월일,가구수,동수,사용승인년월일,거래가,지역,나이,최단지하철역,역과거리,역사명,근접노선수,역lati,역longi,도보거리
10115792,10233309,충북혁신도시아모리움내안애,충청북도 진천군 덕산읍,아파트,36.900717,127.534146,111.32,19.0,2023-02-11,842,13,2018-03-13,35000.0,충북 진천군,6.332649,없음,10000000.0,없음,0,51.5074,0.1278,False
10115793,10233310,충북혁신도시아모리움내안애,충청북도 진천군 덕산읍,아파트,36.900717,127.534146,111.32,12.0,2021-09-03,842,13,2018-03-13,45000.0,충북 진천군,6.332649,없음,10000000.0,없음,0,51.5074,0.1278,False
10115794,10233311,충북혁신도시아모리움내안애,충청북도 진천군 덕산읍,아파트,36.900717,127.534146,111.32,7.0,2021-07-21,842,13,2018-03-13,29500.0,충북 진천군,6.332649,없음,10000000.0,없음,0,51.5074,0.1278,False
10115795,10233312,도운,경상남도 진주시 하대동,아파트,35.188657,128.128217,60.84,1.0,2013-02-15,115,2,1986-07-14,6000.0,경남 진주시,37.995893,가야대,67635.852051,가야대,1,35.266727,128.86507,False
10115796,10233313,도운,경상남도 진주시 하대동,아파트,35.188657,128.128217,60.84,1.0,2012-12-18,115,2,1986-07-14,6000.0,경남 진주시,37.995893,가야대,67635.852051,가야대,1,35.266727,128.86507,False


In [ ]:
df_final.drop(columns=['역사명'])
df_final.to_csv("전국_도시철도역사정보_좌표포함_지하철거리계산.csv", index=False)

# 모델 학습


In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from datetime import datetime
import wandb
from qrytool import load_data_into_dataframe, insert_dataframe_into_table
from geopy.distance import geodesic
import pandas as pd
import concurrent.futures
import os
import ray
from tqdm.notebook import tqdm

# if not df_final :
df_final = pd.read_csv("전국_도시철도역사정보_좌표포함_지하철거리계산.csv")

In [50]:
df_final['거래년월일'] = pd.to_datetime(df_final['거래년월일'])
df_final['사용승인년월일'] = pd.to_datetime(df_final['사용승인년월일'])

In [51]:
# Feature extraction from '거래일' if necessary
df_final['거래연'] = df_final['거래년월일'].dt.year
df_final['거래월'] = df_final['거래년월일'].dt.month
df_final['거래일'] = df_final['거래년월일'].dt.day

df_final['사용승인연'] = df_final['사용승인년월일'].dt.year
df_final['사용승인월'] = df_final['사용승인년월일'].dt.month
df_final['사용승인일'] = df_final['사용승인년월일'].dt.day
# Select features and target
features = ['아파트명', '동네', '건물종류', '위도', '경도', '층', '거래연', '거래월', '거래일', '사용승인연', '사용승인월', '사용승인일', '가구수', '동수', '나이', '역과거리', '근접노선수', '역lati', '역longi', '도보거리']

X = df_final[features]
y = df_final['거래가']

In [52]:
X.shape

(10115797, 20)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import DMatrix, train
from xgboost.callback import TrainingCallback
import wandb
import joblib

# Close any previous W&B runs
wandb.finish()

# Initialize W&B
wandb.init(project="xgboost-best-mon", name="xgboost-best")

# Ensure the target variable does not contain NaN, infinity, or values too large
y = y.replace([np.inf, -np.inf], np.nan)

# Drop corresponding rows in X where y is NaN
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ColumnTransformer for preprocessing
numeric_features = ['위도', '경도', '층', '거래연', '거래월', '거래일', '사용승인연', '사용승인월', '사용승인일',
                    '가구수', '동수', '나이', '역과거리', '근접노선수', '역lati', '역longi']
categorical_features = ['아파트명', '동네', '건물종류', '도보거리']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the training and test data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

joblib.dump(preprocessor, 'feature_preprocessor.pkl')

# Convert to DMatrix
dtrain = DMatrix(X_train_transformed, label=y_train)
dtest = DMatrix(X_test_transformed, label=y_test)

# Set up the parameters for training with the best hyperparameters
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'colsample_bytree': 0.7,
    'learning_rate': 0.2,
    'max_depth': 10,
    'alpha': 100,
    'device': 'cuda'
}


# Custom callback for logging metrics to W&B
class WandbCallback(TrainingCallback):
    def __init__(self):
        self.epoch = 0

    def after_iteration(self, model, epoch, evals_log):
        for eval_set in evals_log:
            for metric in evals_log[eval_set]:
                value = evals_log[eval_set][metric][-1]
                wandb.log({f"{eval_set}_{metric}": value, "epoch": self.epoch})
        self.epoch += 1
        return False  # Return False to continue training


# Train the model and log metrics with a custom callback
num_boost_round = 300
evals = [(dtest, 'eval'), (dtrain, 'train')]

bst = train(params, dtrain, num_boost_round, evals=evals,
            early_stopping_rounds=10, verbose_eval=False,
            callbacks=[WandbCallback()])

# Make predictions using predict with the DMatrix format for GPU
predictions = bst.predict(dtest)

# Calculate Mean Absolute Error as an example metric
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error: ", mae)

# Log the final MAE and save the model to W&B
wandb.log({"Mean Absolute Error": mae})
bst.save_model("xgboost_model.json")
wandb.save("xgboost_model.json")

# Extract feature importance
feature_importance = bst.get_score(importance_type='weight')
print("Feature importance from model:")
print(feature_importance)
# Get the feature names from the model
# model_feature_names = bst.feature_names
# print("Model feature names:")
# print(model_feature_names)
# if model_feature_names is not None:
#     feature_importance = {model_feature_names[int(k[1:])]: v for k, v in feature_importance.items()}


# Get the feature names
encoder = preprocessor.named_transformers_['cat']
categorical_feature_names = encoder.get_feature_names_out(categorical_features)
feature_names = np.concatenate([numeric_features, categorical_feature_names])
print("feature_names", feature_names)
# Create a dictionary to map model feature names to preprocessor feature names
feature_name_mapping = {f'f{i}': feature_names[i] for i in range(len(feature_names))}
print("Feature name mapping:")
print(feature_name_mapping)

# Map the feature importance to the preprocessor feature names
mapped_feature_importance = {feature_name_mapping.get(f, f): imp for f, imp in feature_importance.items()}
print("Mapped feature importance:")
print(mapped_feature_importance)

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({
    'feature': list(mapped_feature_importance.keys()),
    'importance': list(mapped_feature_importance.values())
}).sort_values(by='importance', ascending=False)
# Debug print
print("Feature importance data frame:")
print(importance_df)

# Aggregate importance at the field level
field_importance = {}
for feature, importance in zip(importance_df['feature'], importance_df['importance']):
    field = feature.split('_')[0] if '_' in feature else feature
    if field in field_importance:
        field_importance[field] += importance
    else:
        field_importance[field] = importance


# Create a DataFrame for field-level importance
field_importance_df = pd.DataFrame({
    'field': field_importance.keys(),
    'importance': field_importance.values()
}).sort_values(by='importance', ascending=False)

# Debug print
print("Field importance data frame:")
print(field_importance_df)

# Log field-level importance to W&B as a table
field_importance_table = wandb.Table(dataframe=field_importance_df)
wandb.log({"Field Importance Table": field_importance_table})

# Create a field-level importance bar chart
wandb.log({
    "Field Importance Chart": wandb.plot.bar(
        field_importance_table,
        "field",
        "importance",
        title="Field-Level Feature Importance"
    )
})

# Close the current W&B run
wandb.finish()

# XGboost가 GPU와 동작하는 지 점검


In [ ]:
import xgboost as xgb

# Check if GPU is available and XGBoost is compiled with GPU support


def check_xgboost_gpu():
    params = {'tree_method': 'hist', 'device': 'cuda:0'}
    dmatrix = xgb.DMatrix(data=[[1, 2], [3, 4]], label=[1, 2])
    try:
        # Perform a simple training to check for GPU availability
        xgb.train(params, dmatrix, num_boost_round=1)
        print("XGBoost GPU is available and the model is compiled with GPU support.")
    except xgb.core.XGBoostError as e:
        print("XGBoost GPU is not available or not compiled with GPU support.")
        print(e)


check_xgboost_gpu()

# Hyperparameter Tuning


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, DMatrix, train
from xgboost.callback import TrainingCallback
import wandb
import os

# Close any previous W&B runs
wandb.finish()

os.environ["WANDB_NOTEBOOK_NAME"] = "train_complex_data.ipynb"
# Initialize W&B
wandb.init(project="xgboost-opt-mon", name="xgboost-opt")

# Ensure the target variable does not contain NaN, infinity, or values too large
y = y.replace([np.inf, -np.inf], np.nan)

# Drop corresponding rows in X where y is NaN
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ColumnTransformer for preprocessing
numeric_features = ['위도', '경도', '층', '거래연', '거래월', '거래일', '사용승인연', '사용승인월', '사용승인일',
                    '가구수', '동수', '나이', '역과거리', '근접노선수', '역lati', '역longi']
categorical_features = ['아파트명', '동네', '건물종류', '도보거리']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Custom callback for logging metrics to W&B


class WandbCallback(TrainingCallback):
    def __init__(self):
        super().__init__()
        self.epoch = 0

    def after_iteration(self, model, epoch, evals_log):
        for eval_set in evals_log:
            for metric in evals_log[eval_set]:
                value = evals_log[eval_set][metric][-1]
                wandb.log({f"{eval_set}_{metric}": value, "epoch": self.epoch})
        self.epoch += 1
        return False  # Return False to continue training

    def before_training(self, model):
        self.epoch = 0  # Reset epoch counter for each new training session
        return model


# Create the pipeline with XGBRegressor
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    device='cuda',
    eval_metric='rmse',  # Log RMSE by default
    callbacks=[WandbCallback()],
    use_label_encoder=False,
    verbosity=0
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'model__colsample_bytree': [0.3, 0.5, 0.7],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [5, 7, 10],
    'model__alpha': [1, 10, 100],
    'model__n_estimators': [100, 200, 300]
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    n_jobs=2,
    scoring='neg_mean_absolute_error',
    verbose=2
)

# Fit the model with GridSearchCV
grid_search.fit(X_train, y_train)

# Retrieve the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Transform the test data using the preprocessor
X_test_transformed = best_model.named_steps['preprocessor'].transform(X_test)

# Convert transformed data to DMatrix and ensure it's on the correct device
dtest = DMatrix(X_test_transformed, label=y_test)

# Make predictions using predict with the DMatrix format for GPU
predictions = best_model.named_steps['model'].predict(X_test_transformed)

# Calculate Mean Absolute Error as an example metric
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error: ", mae)

# Log the final MAE and save the model to W&B
wandb.log({"Mean Absolute Error": mae, "Best Params": best_params})
best_model.named_steps['model'].save_model("xgboost_model.json")
wandb.save("xgboost_model.json")

# Close the current W&B run
wandb.finish()

# 모델 inference


## 데이터 준비


In [25]:
from tqdm.notebook import tqdm
import ray
import os
import concurrent.futures
from geopy.distance import geodesic
import wandb
from datetime import datetime
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from qrytool import load_data_into_dataframe, insert_dataframe_into_table
import warnings
import random
import pandas as pd
import json
import numpy as np
import sys
import re
print(sys.path)


def split_address(address):
    match = None
    group2 = None
    group3 = None
    group4 = None
    group5 = None
    group6 = None
    addr1 = None
    addr2 = None
    if pd.notnull(address):
        pattern = re.compile(
            r"\s[\-0-9]+\s(\([가-힣a-zA-Z0-9,\s]+\)\s)?(.*)|.*구.*동\s(.*\s아파트)(.*)|[\s로길동]+[\-0-9번지]+(\([가-힣a-zA-Z0-9,\s]+\))?([,\s].*)"
        )
        match = pattern.search(address)  # search를 사용하여 전체 문자열에서 패턴 매치

        if match:
            group2 = match.group(2)
            group3 = match.group(3)
            group4 = match.group(4)
            group5 = match.group(5)
            group6 = match.group(6)
            # print(
            #     f"{ match.group(1)} | {group2}|{group3}|{group4}|{group5}|{group6}|"
            # )
            if group2:
                addr1 = address.replace(group2, "")
                addr2 = group2
            elif group3:
                if "아파트" in group3:
                    addr1 = address.replace(group4, "")
                    addr2 = group4
                else:
                    addr1 = address.replace(group3, "")
                    addr2 = group3
            elif group6.strip():
                addr1 = address.replace(group6, "")
                addr2 = group6.replace(",", "").strip()
            else:
                addr1 = address
                addr2 = None
            ret1 = addr1
            ret2 = addr2
        else:
            print("No match :" + address)
            ret1 = address
            ret2 = None
    return ret1, ret2


def get_floor(room_number):
    if not room_number:
        return ""
    # Check if the room number is a string
    if isinstance(room_number, str):
        # Check if the string starts with "비"
        if room_number.startswith("B") or room_number.startswith("b"):
            # Remove "비" from the start of the string
            room_number = room_number[1:]
            # Indicate that this is a basement floor
            floor_indicator = '지하'
        else:
            floor_indicator = ''
        # Remove the string "호" from the room number
        room_number = room_number.replace("호", "")

        # Check if the room number is a digit
        if room_number.isdigit():
            # Take all but the last two digits to get the floor number
            floor_number = room_number[:-2]
            return floor_indicator + str(floor_number) + '층'
        else:
            return "Invalid room number"
    else:
        return ""


def is_same_str_by_rem_space(a, b):
    a = a.strip()
    b = b.replace(' ', '').strip()
    if len(a) == len(b) and a == b:
        return a


def has_end_only_braket_string(s):
    # "("가 없고, ")"가 하나만 있으며, ")"로 끝나는지 확인
    return s.count('(') == 0 and s.count(')') == 1 and s.endswith(')')


def has_only_one_set_braket_string(s):
    return s.count('(') == 1 and s.count(')') == 1 and s.startswith('(') and s.endswith(')')


def has_middle_bracket(s):
    return ")" in s and not s.endswith(")") and not s.startswith(")")


def is_single_word(s):
    # 남은 문자열 내에 XXX도 또는 XXX호가 보이지 않는 영문,숫자,한글이 조합된 문자열이라면 True
    s = s.replace(' ', '')
    pattern = r'([\dA-Za-z가나다라마바]+동)*\s*(\d+[호])*'
    match = re.search(pattern, s)
    # match가 None이면 정규식과 일치하는 부분이 없는 것이므로 False 반환
    if match and match.group():
        return False
    else:
        return bool(re.match(r'^[가-힣A-Za-z0-9]+$', s))
    # return bool(re.match(r'^[가-힣A-Za-z]+$', s))


def has_not_relevant_pattern(s):
    pattern = r"^[\s\d\-]+$|^[\s\d]+층\s*$"
    match = re.search(pattern, s)
    if match:
        return True


def get_complex_name(remained_str):
    remained_str = remained_str.strip()
    if remained_str == "":
        return "", ""
    if remained_str[0] == "(":
        remained_str = remained_str[1:].strip()

    if has_only_one_set_braket_string(remained_str):
        return remained_str.replace("(", "").replace(")", "").strip(), ""
    elif has_end_only_braket_string(remained_str):
        return remained_str.replace(")", "").strip(), ""
    elif has_not_relevant_pattern(remained_str):
        print("예외문자열:" + remained_str)
        return "", remained_str
    elif has_middle_bracket(remained_str):
        # print("괄호포함문자열:" + remained_str)
        names = remained_str.split(sep=")")
        if len(names) > 2:
            print("이상한문자열:" + remained_str)
        remained_str = "" if is_same_str_by_rem_space(names[0], names[1]) else names[1].strip()
        return names[0].strip(), remained_str
        # return get_super_str(names[0].strip(), names[1].strip())
    elif is_single_word(remained_str):
        # print("문자열만 남은 경우:", remained_str)
        return remained_str, ""

    else:
        print("추가고려 필요 ==> " + remained_str)
        return "", remained_str


def get_dong_complex_name(remained_str):
    dong_name = ''
    complex_name = ''
    sep = ''
    pattern = r'\(([가-힣\d]+[동가리]{1})([\s,)]){1}'
    if not remained_str or remained_str.strip() == "":
        return "", "", ""
    match = re.search(pattern, remained_str)
    if match:
        dong_name = match.group(1)  # 동명 추출
        sep = match.group(2)  # 동명 추출
        remained_str = remained_str.replace('(' + dong_name + sep, '').strip()
        complex_name, remained_str = get_complex_name(remained_str)
    else:
        complex_name, remained_str = get_complex_name(remained_str)
    return dong_name, complex_name, remained_str


def get_complex_dong_ho_floor(address, lat, lon):
    addr1, addr2 = split_address(address)

    # print(addresses_df.head())
    patterns = [
        r'([\dA-Za-z가나다라마바]+동)\s*(\w+[호]{0,1})\s*$',
        r'\)\s*(\w+)\s*[-ㅡ]\s*([\dA-Za-z]+호*)\s*$',
        r'\)\s*[가-힣A-Za-z\.\,]+\s*([\dABCDEabcde]*)\s*[-ㅡ\s]{1}\s*([\dA-Za-z]+호*)\s*$',
        r'\s*(\w+)\s*[-ㅡ]\s*([\dA-Za-z]+호*)\s*$',
        r'(아파트동)\s*(\w+[호]{0,1})\s*$',
        r'(오피스텔동)\s*(\w+[호]{0,1})\s*$',
        r'\)\s*([\dA-Za-z]+호)\s*$',
        r'\)\s*[가-힣A-Za-z\.\,]+([\dA-Za-z]+호)\s*$',
        r'\s*([\dA-Za-z]+호)\s*$',
    ]
    dong = ""
    ho = ""
    temp_addr2 = ""
    dong_name = ""
    complex_name = ""
    for idx, pattern in enumerate(patterns):
        if not addr2 or addr2.strip() == "":
            continue
        # print(row['회원주소2'])
        match = re.search(pattern, addr2)
        if match:
            if len(match.groups()) == 1:
                ho = match.group(1)  # 호만 추출될 경우
                temp_addr2 = addr2.replace(ho, '')
                if '호' not in ho:
                    ho = ho + '호'
            elif len(match.groups()) >= 2:
                dong = match.group(1)  # 동 추출
                ho = match.group(2)  # 호 추출
                addr2 = addr2.replace(ho, '')
                temp_addr2 = addr2.replace(dong, '')
                if '호' not in ho:
                    ho = ho + '호'
                if len(dong) >= 1 and '동' not in dong:
                    dong = dong + '동'
                if idx in [1, 2, 3]:
                    temp_addr2 = temp_addr2.replace('-', '')
            break
    temp_addr2 = temp_addr2.strip()
    dong_name, complex_name, remained_str = get_dong_complex_name(temp_addr2)

    return complex_name, dong, ho, get_floor(ho)


def calculate_min_distance(row, df_subway):
    lat1, lon1 = row['위도'], row['경도']
    min_distance = float('inf')
    nearest_station = None

    for _, subway_row in df_subway.iterrows():
        lat2, lon2 = subway_row['lati'], subway_row['longi']
        distance = geodesic((lat1, lon1), (lat2, lon2)).meters

        if distance < min_distance:
            min_distance = distance
            nearest_station = subway_row

    return {
        'index': row['index'],
        '최단지하철역': nearest_station['역사명'],
        '역과거리': min_distance,
        '역사명': nearest_station['역사명'],
        '근접노선수': nearest_station['근접노선수'],
        '역lati': nearest_station['lati'],
        '역longi': nearest_station['longi'],
        '도보거리': min_distance <= 500
    }


def get_address_by_name_mdn(name, mdn):
    qry_name_mdn = f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE 이름='{name}' and 결제전화 like '%{mdn}'"
    df = load_data_into_dataframe(qry_name_mdn)
    return df


def get_address_by_cid(cid):
    qry_cid = f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE all_id={cid}"
    df = load_data_into_dataframe(qry_cid)
    return df


def get_address_by_addr(addr):
    qry_cid = f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE 회원주소={addr}"
    df = load_data_into_dataframe(qry_cid)
    return df

['/home/max/miniconda3/lib/python3.10/site-packages/ray/thirdparty_files', '/home/max/cleanbeding/naver-realestate/TIPS', '/home/max/miniconda3/lib/python310.zip', '/home/max/miniconda3/lib/python3.10', '/home/max/miniconda3/lib/python3.10/lib-dynload', '', '/home/max/miniconda3/lib/python3.10/site-packages']


In [56]:
qry = """
SELECT DISTINCT ON (cm.이름, cm.결제전화)
    nc.complex_no,
    nc.complex_name 아파트명,
    nc.cortar_address 동네,
    nc.real_estate_type_name 건물종류,
    nc.latitude 위도,
    nc.longitude 경도,
    0 층,
    nc.use_approve_ymd 사용승인년월일,
    nc.total_household_count 가구수,
    nc.total_building_count 동수,
    nc.cortar_address naver주소,
    CURRENT_DATE 거래년월일,
    cm.all_id,cm.이름,cm.결제전화,cm.회원주소,cm.회원주소1,cm.회원lati,cm.회원longi,cm.가입일시 FROM customatrix cm
LEFT JOIN alladdr_naver_joins nc ON  cm.회원lati=nc.lati AND cm.회원longi=nc.longi
WHERE complex_no IS NOT NULL ORDER BY cm.이름, cm.결제전화;
"""


df = load_data_into_dataframe(qry)

df['거래년월일'] = pd.to_datetime(df['거래년월일'], errors='coerce')
df['사용승인년월일'] = pd.to_datetime(df['사용승인년월일'], errors='coerce')
# df['층'] = df['건물호'].apply(get_floor), df['건물층'])


# Process '동네' column
df['지역'] = df['동네'].apply(process_region_naver_cortar_addr)

# Apply the function to each row in the dataframe
results = df.apply(lambda row: get_complex_dong_ho_floor(row['회원주소'], row['회원lati'], row['회원longi']), axis=1)

# Split the results into separate columns and add them to the dataframe
df[['complex_name', '동호', '호', '층']] = pd.DataFrame(results.tolist(), index=df.index)


# 오늘 날짜를 기준으로 건물의 나이 계산
today = datetime.now()
df['나이'] = (today - df['사용승인년월일']).dt.days / 365.25  # 일수를 년도로 변환
df.reset_index(inplace=True)
df = df[~df['지역'].str.contains('제주')]
df.shape

예외문자열:13층
추가고려 필요 ==> 리젠빌(3차))
추가고려 필요 ==> 절영아파트(3차))
추가고려 필요 ==> 힐스테이트 미사역 그랑파사쥬(11-1BL))
추가고려 필요 ==> 106동
추가고려 필요 ==> 힐스테이트 미사역 그랑파사쥬(11-1BL))
추가고려 필요 ==> 하남미사 롯데캐슬 헤븐시티 Ⅰ
추가고려 필요 ==> 원효루미니아파트 101동(B동)
예외문자열:4층
추가고려 필요 ==> 건설기술인회관(별관))
추가고려 필요 ==> 건설기술인회관(별관))
추가고려 필요 ==> 102동 오피스텔
추가고려 필요 ==> 강변그대家(가) River View(리버 뷰))
추가고려 필요 ==> 잠실 벨솔레 오피스텔 (방이2동 주민센터))
추가고려 필요 ==> 부개주공아파트(6단지))


(2210, 26)

## 지하철 좌표정보와 해당 주소지의 거리를 계산하여 다음의 피처들을 계산해내고 이를 csv 파일에 저장해두는 로직

'최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리'


In [37]:
# 순차적으로 처리 실행
batch_size = 10
num_cores = 23
# 결과 저장할 CSV 파일 초기화
output_file = 'output_inference.csv'


if not ray.is_initialized():
    ray.init(num_cpus=23)


@ray.remote
def process_batch(batch, df_subway):
    results = []
    for _, row in batch.iterrows():
        result = calculate_min_distance(row, df_subway)
        results.append(result)
    return results


# 데이터 로드
file_path = "./전국_도시철도역사정보_좌표포함.xlsx"
df_subway = pd.read_excel(file_path)
df_subway = prepare_subway_data(df_subway)

# 처리된 인덱스 추적
completed_indices = set()
if os.path.exists(output_file):
    processed_results = pd.read_csv(output_file)
    completed_indices = set(processed_results['index'])
else:
    # 결과 파일에 헤더 작성
    pd.DataFrame(columns=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리']).to_csv(output_file, index=False)

# 처리되지 않은 행들만 선택
remaining_rows = df[~df['index'].isin(completed_indices)]
total_rows = len(remaining_rows)

# Custom bar format for tqdm
bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}] {percentage:3.0f}%'

# Process the data in batches
with tqdm(total=total_rows, desc="Processing rows", bar_format=bar_format) as pbar:
    futures = []
    start_index = 0

    while start_index < total_rows or futures:
        # Submit tasks until the core limit is reached
        while len(futures) < num_cores and start_index < total_rows:
            batch = remaining_rows.iloc[start_index:start_index + batch_size]
            future = process_batch.remote(batch, df_subway)
            futures.append(future)
            start_index += batch_size

        # Wait for the first available future to complete
        if futures:
            ready_futures, remaining_futures = ray.wait(futures, num_returns=1)
            futures = remaining_futures

            for ready_future in ready_futures:
                result = ray.get(ready_future)
                result_df = pd.DataFrame(result)
                result_df.to_csv(output_file, mode='a', header=False, index=False)
                pbar.update(batch_size)

# Shut down Ray
ray.shutdown()

2024-07-12 22:21:27,822	INFO worker.py:1771 -- Started a local Ray instance.


Processing rows: |          | 0/0 [00:00<?, ?it/s]   0%

저장된 지하쳘역 관련 피처들을 기본 학습데이터와 조인


In [57]:
# processed_results = pd.read_csv(output_file, header=None, names=['index', '최단지하철역', '역과거리', '역사명', '근접노선수', '역lati', '역longi', '도보거리'])
processed_results = pd.read_csv(output_file)

# Ensure 'index' is the correct data type
# df.reset_index(inplace=True)
df_merged = df.merge(processed_results, on='index')
df_merged.drop(columns=['역사명'])
df_merged.to_csv("고객주소지기준_학습데이터에_지하철거리계산데이터포함.csv", index=False)
processed_results.shape, df_merged.shape

((2210, 8), (2210, 33))

# 저장된 에측 대상 데이터 로드후 학습


In [58]:
df_final = pd.read_csv("고객주소지기준_학습데이터에_지하철거리계산데이터포함.csv")

# Feature extraction from '거래일' if necessary
df_final = df_merged.copy()

df_final['거래년월일'] = pd.to_datetime(df_final['거래년월일'])
df_final['사용승인년월일'] = pd.to_datetime(df_final['사용승인년월일'])

df_final['거래연'] = df_final['거래년월일'].dt.year
df_final['거래월'] = df_final['거래년월일'].dt.month
df_final['거래일'] = df_final['거래년월일'].dt.day

df_final['사용승인연'] = df_final['사용승인년월일'].dt.year
df_final['사용승인월'] = df_final['사용승인년월일'].dt.month
df_final['사용승인일'] = df_final['사용승인년월일'].dt.day
# '층' 문자를 제거하고 숫자형으로 변환하는 함수 정의


def clean_floor_value(floor):
    if isinstance(floor, str):
        floor = floor.replace('층', '')
        # floor = floor.replace('지하', '-')
        if floor == '' or floor == 'Invalid room number':
            floor = '1'  # 빈 값 또는 'Invalid room number'을 1로 대체
        return int(floor)
    return floor


df_final = df_final[~df_final['층'].str.contains('지하')]
# df_final의 '층' 열에 함수 적용
df_final['층'] = df_final['층'].apply(clean_floor_value)

In [ ]:
df_final['층'].unique()
df_final

## 모델로드후 준비된 데이터로 인퍼런스


In [59]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib
# Display the dataframe with predictions
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

preprocessor = joblib.load('feature_preprocessor.pkl')

# Load the model
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model.json")

# Define the features
numeric_features = ['위도', '경도', '층', '거래연', '거래월', '거래일', '사용승인연', '사용승인월', '사용승인일', '가구수', '동수', '나이', '역과거리', '근접노선수', '역lati', '역longi']
categorical_features = ['아파트명', '동네', '건물종류', '도보거리']
features = numeric_features + categorical_features

# Load or prepare df_final
# Assuming df_final is already loaded and processed
# df_final should have columns matching the features list
# Ensure df_final contains only the relevant features
df_final = df_final[features]
display(df_final.head())
# ColumnTransformer for preprocessing
# numeric_transformer = StandardScaler()
# categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ])

# Fit the preprocessor on the entire dataset (assuming you have a training set available)
# Here you should fit the preprocessor on the same data you used for training the model.
# For demonstration, we're fitting it on df_final.
# X_transformed = preprocessor.fit_transform(df_final)
X_transformed = preprocessor.transform(df_final)


# Convert to DMatrix
dnew = xgb.DMatrix(X_transformed)

# Predict
predictions = loaded_model.predict(dnew)

# Optionally, you can add the predictions to the dataframe
df_final['Predictions'] = predictions

# Merge predictions with df_merged if necessary
# Ensure df_merged is properly loaded and available in your context
df_merged = df_merged.join(df_final[['거래연', '거래월', '거래일', '사용승인연', '사용승인월', '사용승인일', 'Predictions']], how='left')


# Print or return the merged dataframe with predictions
# print(df_merged)

,위도,경도,층,거래연,거래월,거래일,사용승인연,사용승인월,사용승인일,가구수,동수,나이,역과거리,근접노선수,역lati,역longi,아파트명,동네,건물종류,도보거리
0,36.476078,127.146003,14,2024,7,12,1994.0,10.0,22.0,312,6,29.724846,34477.676883,1,36.777541,127.052751,한빛,충청남도 공주시 신관동,아파트,False
1,37.268165,127.000106,10,2024,7,12,2006.0,3.0,17.0,260,1,18.324435,223.730997,2,37.266162,126.999821,세진브론즈빌,경기도 수원시 팔달구 매산로1가,오피스텔,True
2,37.456136,126.653394,26,2024,7,12,NaN,NaN,NaN,628,1,NaN,936.417680,1,37.448191,126.649832,인천효성해링턴타워인하,인천시 미추홀구 용현동,오피스텔,False
3,37.532110,127.069120,5,2024,7,12,2009.0,9.0,15.0,279,4,14.825462,219.841698,1,37.531590,127.066720,이튼타워리버5차,서울시 광진구 자양동,아파트,True
4,37.502964,126.856017,16,2024,7,12,1999.0,2.0,24.0,987,13,25.382615,943.409092,1,37.494698,126.858504,고척대우,서울시 구로구 고척동,아파트,False


In [60]:
df_merged.shape

(2210, 40)

In [61]:
insert_dataframe_into_table(df_merged, 'clbe_customer_realestate_model', if_exists='append')